# Question 3

In [478]:
import torch
import torch.nn as nn
import pandas as pd
import os
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

from skimage import io,transform
from torchvision import datasets
from torchvision.transforms import ToTensor, ToPILImage, RandomCrop

%matplotlib inline
import matplotlib.pyplot as plt

In [479]:
df = pd.read_csv("deeplearning/train/labels.csv",header=None)

In [480]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [481]:
def show_face(image, camera_position):
    """Show image with landmarks"""
    plt.title(camera_position)
    plt.imshow(image,cmap="gray")
    plt.pause(0.001)  # pause a bit so that plots are updated


In [482]:
def encode_dataframe(df):
    le = LabelEncoder()
    df[1] = df[1].astype(str) + df[2].astype(str)+ df[3].astype(str)
    df[1]  = le.fit_transform(df[1])
    return df

In [483]:
encode_dataframe(df)

,0,1,2,3
0,0001.jpg,63,0.000000,0.000000
1,0002.jpg,56,0.000000,0.342020
2,0003.jpg,37,0.000000,0.707107
3,0004.jpg,14,0.000000,1.000000
4,0005.jpg,55,0.000000,-0.342020
...,...,...,...,...
1833,1834.jpg,4,-0.719846,0.642788
1834,1835.jpg,2,-0.397131,0.906308
1835,1836.jpg,6,-0.883022,-0.342020
1836,1837.jpg,10,-0.866025,0.000000


In [484]:
class FaceDataset(Dataset):
    """Custom Fashion MNIST dataset."""

    def __init__(self, csv_file, image_dir,):
        self.labels_df = pd.read_csv(csv_file,header=None)
        self.labels_df = encode_dataframe(self.labels_df)
        self.transform = transform
        self.image_dir = image_dir

    def __len__(self):
        return len(self.labels_df)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.image_dir,
                                self.labels_df.iloc[idx, 0])
        image = io.imread(img_name, as_gray=True)
        image = transform.resize(image,(128,128))
        convert_tensor = transforms.ToTensor()
        image = convert_tensor(image)
        image.unsqueeze(0)
        camera_position = self.labels_df.iloc[idx, 1]

        data = {'image': image, 'camera_position': camera_position}




        return image, camera_position

In [485]:
face_dataset = FaceDataset(csv_file='deeplearning/train/labels.csv',
                           image_dir='deeplearning/train')

In [486]:
dataloader = DataLoader(face_dataset, batch_size=4,
                        shuffle=True, num_workers=0)

In [487]:
# Solution

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.convlayers = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=6, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(6),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2), # B x 6 x 64 x 64 after this maxpool
            nn.Conv2d(in_channels=6, out_channels=12, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(12),
            nn.ReLU(),
        )

        self.MLP = nn.Sequential(
            nn.Linear(in_features=12*64*64,out_features=120),
            nn.BatchNorm1d(120),
            nn.ReLU(),
            nn.Linear(in_features=120,out_features=84),
            nn.BatchNorm1d(84),
            nn.ReLU(),
            nn.Linear(in_features=84,out_features=64)
        )

    def forward(self, x):
        # Input x has dimensions B x 1 x 28 x 28, B is batch size
        x = self.convlayers(x)
        x = x.view(x.size(0), -1)
        print(x.shape)
        x = self.MLP(x)
        # Output has dimensions B x 10
        return x

model = CNN()
loss_func = nn.CrossEntropyLoss()

In [488]:
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(total_params)

5915128


In [489]:
images,labels = next(iter(dataloader))

In [490]:
images.shape

torch.Size([4, 1, 128, 128])

In [491]:
labels =  labels.type(torch.LongTensor)
output = model(images.float())
print(output.shape)
loss = loss_func(output,labels)
print(loss)

torch.Size([4, 49152])
torch.Size([4, 64])
tensor(4.3986, grad_fn=<NllLossBackward0>)


In [491]:
num_epochs = 20

# Set up the optimiser
optim = torch.optim.Adam(model.parameters(), lr = 0.1)

# Initialise some variables for computing and tracking stats
iterations_per_epoch = math.ceil(len(train_data)/batch_size)
training_losses = []
training_accuracies = []
testing_losses = []
testing_accuracies = []
for epoch in range(num_epochs):

    # One epoch on the training set
    total_loss = 0
    correct = 0
    total = 0
    total_loss = 0
    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        output = model(inputs)
        loss = loss_func(output,labels)
        optim.zero_grad()
        loss.backward()
        optim.step()
        pred_y = torch.argmax(output, 1)
        correct += (pred_y == labels).sum()
        total += float(labels.size(0))
        total_loss += loss*images.shape[0]
        if (i+1) % 100 == 0:
            print('Epoch [{}/{}], Iteration [{}/{}], Loss: {:.4f}'.format(epoch + 1, num_epochs, i + 1, iterations_per_epoch, loss.item()))
    total_loss /= len(train_data)
    training_losses.append(total_loss.item())
    training_accuracies.append(correct/total)
    print('Train accuracy over epoch {}: {:.4f}'.format(epoch+1,training_accuracies[-1]))

    # One epoch on the test set
    correct = 0
    total = 0
    # Switch to evaluation mode
    model.eval()
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            output = model(inputs)
            loss = loss_func(output,labels)
            pred_y = torch.argmax(output, 1)
            correct += (pred_y == labels).sum()
            total += float(labels.size(0))
            total_loss += loss*images.shape[0]
        test_accuracy = correct/total
    total_loss /= len(test_data)
    testing_losses.append(total_loss.item())
    # Switch back to training mode
    model.train()
    testing_accuracies.append(test_accuracy)
    print('Test accuracy at epoch {}: {:.4f}'.format(epoch+1,test_accuracy))